In [16]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import ast

### **[데이터 로드]**

In [ ]:
"""

embedding: OpenAI text-embedding-ada-002 통해 추출

아래 파일 구글 드라이브 다운로드.
spnews_summary_norm.csv = 
    summary_df_final.csv + 
    sia_embed_ada.csv + 
    spnews_final.csv

LLMtoDatabase.py 를 통해 db 에 저장된 데이터가 spnews_summary_norm.csv 라고 가정 후 진행.

"""
# 
# 

csv_path = "data/spnews_summary_norm.csv"

### **[정규화]**

In [ ]:
"""

[작성 2025-11-25]
현재 embedding column은 정규화가 되지 않은 상태.
object 를 array 형태로 변환 후 정규화 진행.
(postgres에서는 pgvector를 이용하여 진행하기 때문에, rec.py 에는 정규화 코드가 없음)

[수정 2025-12-01]
정규화된 파일을 "spnews_summary_norm.csv" 로 저장하였으므로, 아래 코드는 실행하지 않음.

"""
# df_summary["embedding"] = df_summary["embedding"].apply(ast.literal_eval)
# embeddings = np.array(df_summary["embedding"].to_list())

In [ ]:
# 정규화 진행
# norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
# emb_norm = embeddings / norms

In [ ]:
# df_summary["embedding"] = emb_norm.tolist()

In [ ]:
# df_summary.to_csv("data/spnews_summary_norm.csv", index=False)

### **[추천 시스템]**

In [ ]:
class RecommenderTest:

    def __init__(self, csv_path):
        """

        [작성 2025-12-01] 
        테스트 목적의 CSV 기반 추천 시스템.
        
        input: 사용자가 선택한 기사 id (click_id)
        output: 관련 기사 

        postgres 없이도 실행 가능하도록 작성하였음.(postgres 버전은 Recommender)     

        [수정 2025-12-01]
        issue: 입력 id를 기준으로 row를 탐색하다보니 시간이 매우 오래걸림.
        해결: idx를 부여하여 빠르게 탐색하도록 수정. (id_to_index 추가)
        
        """

        df = pd.read_csv(csv_path)

        # object > array
        df["embedding"] = df["embedding"].apply(ast.literal_eval)
        embeddings = np.array(df["embedding"].to_list())

        id_to_index = {id_val: idx for idx, id_val in enumerate(df["id"])}

        self.df = df
        self.embeddings = embeddings
        self.id_to_index = id_to_index

    def get_similar_articles(self, click_id, k):
        """

        [작성 2025-12-01]

        click_id: 사용자가 클릭한 기사 id
        k: 추천할 기사의 개수

        click_id의 embedding값을 참고하여 코사인 유사도를 계산.

        반환: json 형식.

        """

        click_idx = self.id_to_index[click_id]
        click_vec = self.embeddings[click_idx]

        # 코사인 유사도 계산
        sims = self.embeddings @ click_vec 

        # 본인 제외
        sims[click_idx] = -1.0

        # 상위 k개 리턴
        top_idx = np.argsort(-sims)[:k]  

        results = []
        for idx in top_idx:
            row = self.df.iloc[idx]

            results.append(
                {
                    "id": row["id"],
                    "category": row["category"],
                    "publish_date": str(row["publish_date"])[:10],
                    "title": row["title"],
                    "url": row["url"]
                }
            )

        return results

### **[테스트]**

In [28]:
%%time

test = RecommenderTest(csv_path = csv_path)

similar = test.get_similar_articles("spnews_101404", k=10)

for article in similar:
    print(article)

{'id': 'spnews_50397', 'category': '경제/산업', 'publish_date': '2022-03-22', 'title': '함남 연포온실농장건설...성토작업-기초공사 한창', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=50397'}
{'id': 'spnews_54220', 'category': '경제/산업', 'publish_date': '2022-07-18', 'title': '함경남도 연포온실농장, 온실타일붙이기·살림집 건설', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=54220'}
{'id': 'spnews_55668', 'category': '경제/산업', 'publish_date': '2022-09-05', 'title': '北 "화성지구 살림집·연포온실농장·검덕지구 살림집 공사 한창"', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=55668'}
{'id': 'spnews_50248', 'category': '경제/산업', 'publish_date': '2022-03-16', 'title': '평양시 화성지구 살림집건설·함남 연포온실농장건설 한창', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=50248'}
{'id': 'spnews_48611', 'category': '정치', 'publish_date': '2022-01-28', 'title': '北 김정은, 함남 함주군 연포지구 대규모 채소온실 건설예정지 시찰', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=48611'}
{'id': 'spnews_73131', 'category': '경제/산업', 'publish_date': '2023-11-17